## 爬取今天新聞的url連結

In [2]:
import requests
from bs4 import BeautifulSoup
from urlparse import urljoin
import os
import datetime
import time
import math
import json

today = datetime.datetime.now()
fileName = today.strftime('%Y%m%d')
today = today.strftime('%Y%m')

#全域變數
urlString='' #要印出的url長串
main_url ='http://udn.com/' #結合要用的url

#要爬的新聞類別陣列
myList=['6638','6648','7227','7225','6639','6644','6649','6641','6640','7226']
for i in myList:
    url = 'http://udn.com/rank/pv/2/{}/1'.format(i)
    res = requests.get(url)
    #####先指定編碼#####
    res.encoding = "utf-8"
    #####先指定編碼#####
    soup = BeautifulSoup(res.text)
    dts = soup.select('#breaknews_body dl dt')
    d1s = soup.select('.info.only_web')


    for dt in dts:
        try:
    #         印出該dt是甚麼class，傳回來的會是 list 
    #         print(dt.get('class'))

            #抓到類別 
            if len(dt.select('.cate')) != 0:
                category = dt.select('.cate')[0].text
    #             print(category)


            #抓到url & title
            if len(dt.select('h2 a')) != 0:
                href = dt.select('h2 a')[0]['href']
                title =dt.select('h2 a')[0].text.strip()
                news_url = urljoin(main_url,href)
    #             print(news_url)
    #             print(title)

            #抓到日期
            if dt.get('class')[0] == 'dt':
                date1 = dt.text.split(' ')[0].split('-')[1]
                date = today + date1
    #             print(date)

            #抓到人氣
            if dt.get('class')[0] == 'only_web':
                click = dt.text
    #             print(click)

                if date == fileName:       
                    urlString += news_url+','+category+','+title+','+date+','+click+'\n'
    #                 print(urlString)
        except:
            continue
    time.sleep(2)    
f = open('C:/Users/ytchen/Desktop/uniNews/{}.txt'.format(fileName),'w')
f.write(urlString.encode('utf-8')) #最後再一次寫出來
f.close()             

C:\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


## 爬取內文

In [1]:
import datetime
import requests
import time
from bs4 import BeautifulSoup
import json
import time

#全域變數
error_count = 0
error_url = ''
all_json_str = [] #存所有資訊的 json

#抓日期
today = datetime.datetime.now()
today = today.strftime('%Y%m%d')
fileName = today

#開起每天的url檔案
f = open('C:/Users/ytchen/Desktop/uniNews/{}.txt'.format(today),'r')
for url_no in f.readlines():
    try:
        myList = url_no.split(',')
        url = myList[0]
        category =  myList[1]
        title =  myList[2]
        date =  myList[3]
        click =  myList[4].replace('\n','')
#         print(url,category,title,date,click)
        
        #發送request
        res = requests.get(url) #利用get發送請求
        res.encoding = "utf-8" #先進行編碼
        soup = BeautifulSoup(res.text)
        
        #抓內文
        article = ''
        content= soup.select('p')   #先把所有內文抓到
        for p in content:           #把每個p標籤一個一個抓出來印
            article += p.text.replace('\r','').replace('\n','').strip() #段落會隱藏換行符號\r，要取代掉，不然後面讀取json格式會只讀到最後一行
        
        #抓keyword
        keywords = soup.select('#story_tags a')
        keyword = []
        for k in keywords:
            keyword.append(k.text)
            
        #將資訊存成 json檔
        data = { 
            'comp':'uni',  #新聞來源
            'category':category, #類別
            'title':title, #標題
            'content':article, #內文
            'date':date,  #直接抓檔案的日期來用
            'url':url,
            'keyw':keyword,
            'click':click
        }
        
        #一篇新聞一本字典
        all_json_str.append(data) 
        time.sleep(1)
        
    except:                     
        error_count += 1
        error_url += url+','
        continue
        
time.sleep(1.5)
#檔案儲存
f1 = open('C:/Users/ytchen/Desktop/uniNews/json/{}.json'.format(fileName),'w')
b =json.dumps(all_json_str) #再把 list 字典轉換成 json 並包起來
f1.write(b)

#寫入檔案結束
f1.close()     

#log記錄
logString = ''
fileName_log = fileName+'/'+'errorCount:'+str(error_count)+'\n'+'error_url:'+error_url+'\n'
#讀取
with open('C:/Users/ytchen/Desktop/uniNews/log_content.txt','r') as f1:
    for logLine in f1.readlines():
        logString += logLine
#重新寫入
with open('C:/Users/ytchen/Desktop/uniNews/log_content.txt','w') as f2:
    logString += fileName_log
    f2.write(logString.encode('utf-8'))

D:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


## 將json倒入MongoDB

In [2]:
import pymongo
from pymongo import MongoClient
import json
import datetime

#抓日期
today = datetime.datetime.now()
today = today.strftime('%Y%m%d')
fileName = today
print fileName

f = open('C:/Users/ytchen/Desktop/uniNews/json/{}.json'.format(fileName),'r')
i = f.read()
json = json.loads(i)
    
client = MongoClient() 
database = client["test"]
collection = database["news"]
collection.insert_many(
    json
)
client.close()
f.close()

20161011
